In [199]:
import pandas as kungfupanda
from os import listdir
from os.path import isfile, join
import os
import re

In [200]:
leetcodePath = 'my-submissions/'
leetcodeFiles = [x for x in listdir(leetcodePath)]

print(leetcodeFiles)
print(f'Total of {len(leetcodeFiles)} files found.')


['e1.java', 'e1068.sql', 'e118.py', 'e1184.py', 'e14.java', 'e175.sql', 'e181.sql', 'e182.sql', 'e183.sql', 'e197.sql', 'e20.java', 'e21.java', 'e2248.py', 'e2395.py', 'e26.java', 'e2678.java', 'e28.py', 'e2886.py', 'e2894.py', 'e292.py', 'e3074.py', 'e345.py', 'e414 - sorting.py', 'e414.py', 'e577.sql', 'e584.sql', 'e586.sql', 'e595.sql', 'e596.sql', 'e607.sql', 'e619.sql', 'e70 - bottomup.py', 'e70 - topdown.py', 'e9.java', 'm1218.py', 'm1404.py', 'm1442.py', 'm166.py', 'm176.sql', 'm177.sql', 'm1885.py', 'm2.java', 'm2433.py', 'm251.py', 'm260.py', 'm2679.java', 'm3.java', 'm611.py', 'm651.py', 'm7.py']
Total of 50 files found.


In [201]:
leetcodeFiles.sort()

In [202]:
# def addCase(df: kungfupanda.DataFrame,
def addCase(level: str, 
            number: int, 
            title: str, 
            language: str,
            path: str) :
    # template = {'Level': level, 'Number': number, 'Title': title, 'Python': '', 'Java': '', 'MySQL': '', 'Other': ''}
    # template[language] = path

    # Level, Number, Title, Python, Java, MySQL, Other
    output = [level, number, title, '', '', '', '']
    path = f'[{language}](<{path}>)'
    
    match language.lower() :
        case 'python' | 'py':
            output[3] = path
        case 'java':
            output[4] = path
        case 'mysql' | 'sql' :
            output[5] = path
        case _:
            output[6] = path

    # return df
    return output

In [203]:
def updateLanguage(orig, language, path) :    
    match language.lower() :
        case 'python' | 'py':
            if not orig[3] == '' :
                print(f'Warning: Python solution already exists for {orig[2]}.')
                return updateLanguage(orig, 'other', path)
            
            path = f'[{language}](<{path}>)'
            orig[3] = path
        case 'java':
            if not orig[4] == '' :
                print(f'Warning: Java solution already exists for {orig[2]}.')
                return updateLanguage(orig, 'other', path)
            
            path = f'[{language}](<{path}>)'
            orig[4] = path
        case 'mysql' | 'sql' :
            if not orig[5] == '' :
                print(f'Warning: MySQL solution already exists for {orig[2]}.')
                return updateLanguage(orig, 'other', path)
            
            path = f'[{language}](<{path}>)'
            orig[5] = path
        case _:
            if not orig[6] == '' :
                print(f'Warning: Other solution already exists for {orig[2]} and is being overwritten.')
            
            path = f'[{language}](<{path}>)'
            orig[6] = path

    return orig

In [204]:
easyQuestions = [] 
mediumQuestions = []
hardQuestions = []

questionsVisited = set()

for i in leetcodeFiles :
    level = i[0].lower()
    number = int(re.sub("[^0-9]", "", i))
    title = ''
    language = i[i.find('.') + 1:]
    path = join(leetcodePath, i)

    # print(level, number, title, language, path)

    # match level :
    #     case 'e' :
    #         dfEasy = addCase(dfEasy, 'Easy', number, title, language, path)
    #     case 'm' :
    #         dfMedium = addCase(dfMedium, 'Medium', number, title, language, path)
    #     case 'h' :
    #         dfHard = addCase(dfHard, 'Hard', number, title, language, path)

    if number in questionsVisited :
        match level :
            case 'e' :
                for i in range(len(easyQuestions)) :
                    if easyQuestions[i][1] == number :
                        easyQuestions[i] = updateLanguage(easyQuestions[i], language, path)
                        break
            case 'm' :
                for i in range(len(mediumQuestions)) :
                    if mediumQuestions[i][1] == number :
                        mediumQuestions[i] = updateLanguage(mediumQuestions[i], language, path)
                        break
            case 'h' :
                for i in range(len(hardQuestions)) :
                    if hardQuestions[i][1] == number :
                        hardQuestions[i] = updateLanguage(hardQuestions[i], language, path)
                        break
        continue

    questionsVisited.add(number)

    match level :
        case 'e' :
            easyQuestions.append(addCase('Easy', number, title, language, path))
        case 'm' :
            mediumQuestions.append(addCase('Medium', number, title, language, path))
        case 'h' :
            hardQuestions.append(addCase('Hard', number, title, language, path))

print(easyQuestions)
print(mediumQuestions)
print(hardQuestions)


[['Easy', 1, '', '', '[java](<my-submissions/e1.java>)', '', ''], ['Easy', 1068, '', '', '', '[sql](<my-submissions/e1068.sql>)', ''], ['Easy', 118, '', '[py](<my-submissions/e118.py>)', '', '', ''], ['Easy', 1184, '', '[py](<my-submissions/e1184.py>)', '', '', ''], ['Easy', 14, '', '', '[java](<my-submissions/e14.java>)', '', ''], ['Easy', 175, '', '', '', '[sql](<my-submissions/e175.sql>)', ''], ['Easy', 181, '', '', '', '[sql](<my-submissions/e181.sql>)', ''], ['Easy', 182, '', '', '', '[sql](<my-submissions/e182.sql>)', ''], ['Easy', 183, '', '', '', '[sql](<my-submissions/e183.sql>)', ''], ['Easy', 197, '', '', '', '[sql](<my-submissions/e197.sql>)', ''], ['Easy', 20, '', '', '[java](<my-submissions/e20.java>)', '', ''], ['Easy', 21, '', '', '[java](<my-submissions/e21.java>)', '', ''], ['Easy', 2248, '', '[py](<my-submissions/e2248.py>)', '', '', ''], ['Easy', 2395, '', '[py](<my-submissions/e2395.py>)', '', '', ''], ['Easy', 26, '', '', '[java](<my-submissions/e26.java>)', '', '

In [205]:
# Sorting by question number
easyQuestions = sorted(easyQuestions, key=lambda x: x[1])
mediumQuestions = sorted(mediumQuestions, key=lambda x: x[1])
hardQuestions = sorted(hardQuestions, key=lambda x: x[1])

In [206]:
columns = ['Level', 
           'Number', 
           'Title', 
           'Python', 
           'Java', 
           'MySQL', 
           'Other'
           ]
typeClarification = {'Level': 'str', 
                     'Number': 'int', 
                     'Title': 'str', 
                     'Python': 'str', 
                     'Java': 'str', 
                     'MySQL': 'str', 
                     'Other': 'str'
                     }

dfEasy = kungfupanda.DataFrame(data=easyQuestions, columns=columns)
dfMedium = kungfupanda.DataFrame(data=mediumQuestions, columns=columns)
dfHard = kungfupanda.DataFrame(data=hardQuestions, columns=columns)

dfEasy = dfEasy.astype(typeClarification)
dfMedium = dfMedium.astype(typeClarification)
dfHard = dfHard.astype(typeClarification)

# print(dfEasy)
# print(dfMedium)
# print(dfHard)

# print(dfEasy.to_markdown(index=False))

In [207]:
readmePath = 'README_test.md'

with open(readmePath, 'w') as file :
    file.write('# LeetCode\n\n')

    file.write(f'Easy completed:\t{len(easyQuestions)}\n')
    file.write(f'Medium completed:\t{len(mediumQuestions)}\n')
    file.write(f'Hard completed:\t{len(hardQuestions)}\n\n')
    
    file.write('## Easy\n')
    file.write(dfEasy.to_markdown(index=False))

    file.write('\n\n## Medium\n')
    file.write(dfMedium.to_markdown(index=False))

    file.write('\n\n## Hard\n')
    file.write(dfHard.to_markdown(index=False))

# readme = os.open(readmePath, os.O_WRONLY|os.O_CREAT)

# os.write(readme, b'# LeetCode\n\n')

# os.write(readme, b'## Easy\n')
# os.write(readme, dfEasy.to_markdown(index=False).encode())

# os.write(readme, b'\n\n## Medium\n')
# os.write(readme, dfMedium.to_markdown(index=False).encode())

# os.write(readme, b'\n\n## Hard\n')
# os.write(readme, dfHard.to_markdown(index=False).encode())

# os.close(readme)